# Libraries

In [3]:
!pip3 install arch
!pip3 install pandas_ta

from arch import arch_model
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
from IPython.display import clear_output
import pandas_ta as ta
from numpy import nan as npNaN
warnings.filterwarnings('ignore')
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from scipy.signal import welch
from scipy.stats import entropy
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


ImportError: cannot import name 'NaN' from 'numpy' (/Users/krishnagarg/Library/Python/3.9/lib/python/site-packages/numpy/__init__.py)

# Parameters for identification 

In [2]:
def mean_rev_para(close,std=0.45):
    close=pd.Series(close)
    result = ts.adfuller(close)
    adf_statistic = result[0]
    p_value = result[1]
    used_lag = result[2]
    n_obs = result[3]
    critical_values = result[4]
    ic_best = result[5]
    if (adf_statistic<critical_values['1%']+std or adf_statistic<critical_values['5%']+std or adf_statistic<critical_values['10%']+std):
        return 1
    else:
        return 0

def adx(close):
    close=pd.Series(close)
    result = ts.adfuller(close)
    p_value = result[1]
    if p_value<0.2:
        return 1
    else: 
        return 0

def high_mean(close) :
    close=pd.Series(close)
    return close.mean() /max(close)
def low_mean(close) :
    close=pd.Series(close)
    return close.mean() /min(close)
def kurto(close) :
    close=pd.Series(close)
    return close.kurtosis()
def skew(close) :
    close=pd.Series(close)
    return close.skew()

In [3]:


def spectral_entropy(time_series, sampling_frequency=1, nperseg=None):
    """
    Compute the spectral entropy of a time series.

    :param time_series: Array-like, the time series data.
    :param sampling_frequency: Sampling frequency of the time series (default=1).
    :param nperseg: Length of each segment for Welch's method (default=None, which means nperseg = 256).
    :return: Spectral entropy value.
    """
    time_series = np.array(time_series)

    # Compute the power spectral density (PSD) using Welch's method
    freqs, psd = welch(time_series, fs=sampling_frequency, nperseg=nperseg)

    # Normalize the PSD to get a probability distribution
    psd_norm = psd / np.sum(psd)

    # Compute the spectral entropy
    spec_entropy = entropy(psd_norm)

    return spec_entropy


In [4]:
def bollinger_cross(close):
    data = pd.DataFrame({
      'Close': close
  })
    
    data['mean'] = data['Close'].rolling(window = 10).mean()
    data['std'] = data['Close'].rolling(window = 10).std()
    data['upper_band'] = data['mean'] + 2*data['std']
    data['lower_band'] = data['mean'] - 2*data['std']
    
    count = 0
    for i in range(len(data)):
        if data['Close'][i] > data['upper_band'][i] and data['Close'][i-1] < data['upper_band'][i-1] :
            count += 1
        elif data['Close'][i] < data['lower_band'][i] and data['Close'][i-1] > data['lower_band'][i-1] :
            count += 1
        
    return count

In [5]:
def garch(close):

    close=pd.Series(close)
    close=close.pct_change().dropna()
    am = arch_model(close, vol='Garch', p=1, o=0, q=1, dist='Normal')
    res = am.fit(disp=  'off')
    params = res.params
    omega = params['omega']*1000  # The constant term
    alpha = params['alpha[1]']  # The GARCH term
    beta = params['beta[1]']  # The ARCH term
    return omega,alpha,beta

In [6]:
def ADX_SCORE(high, low, close):
  # for i in range(len(TIME_PERIOD)):
    # close = TIME_PERIOD.close[i]
    # high = TIME_PERIOD.high[i]
    # low = TIME_PERIOD.low[i]

    data = pd.DataFrame({
        'High': high,
        'Low': low,
        'Close': close
    })
    # print(data)
    ad = ta.adx(data['High'], data['Low'], data['Close'], length=10)
    # print(ad)
    if ad is None:
      ADX_mean = 0
      PDI_mean = 0
      NDI_mean = 0
    else:
      
      data['ADX'] = ad['ADX_10']
      data['+DI'] = ad['DMP_10']
      data['-DI'] = ad['DMN_10']

      ADX_mean = np.mean(data['ADX'])
      PDI_mean = np.mean(data['+DI'])
      NDI_mean = np.mean(data['-DI']) 






    if ADX_mean >= 40 and PDI_mean > NDI_mean:
      return 1
    elif ADX_mean >= 40 and PDI_mean < NDI_mean:
      return -1
    else:
      return 0

In [7]:
def ts_charactertsics(close):
  close = pd.Series(close)
  Mean = np.mean(close)
  Std = np.std(close)
  Var = np.var(close)
  Median = np.median(close)
  return Mean/Std, Mean/Var, Mean/Median

In [8]:
def slope(close):
  data = pd.DataFrame({
      'Close': close
  })
  data['shift_close'] = data['Close'].shift(-1)
  data['SLOPE'] = (data.shift_close - data.Close)*100/data.Close
  data.dropna()
  return np.mean(data.SLOPE)

In [9]:
def shoulder_distance(close):
    y_true = np.array(close)
    n = len(close)
    x = np.arange(n)

    slope = (close[-1] - close[0]) / (n - 1)
    intercept = close[0]


    y_theoretical = slope * x + intercept
    deviation = y_theoretical - y_true
    deviation_abs = np.abs(deviation)

    index_max = np.argmax(deviation_abs)

    index_second_max = np.argpartition(deviation_abs, -2)[-2]

    if deviation[index_max] * deviation[index_second_max] < 0:
        return -abs(index_max - index_second_max)
    else:
        return abs(index_max - index_second_max)


# DATA 

In [10]:
nifty=pd.read_csv(r'https://raw.githubusercontent.com/Akshatgarg221/fac/main/ind_nifty50list%20(2).csv')

In [11]:
nifty

,COMAPANY,SECTOR,Ticker
0,Adani Enterprises Ltd.,Metals & Mining,ADANIENT.NS
1,Adani Ports and Special Economic Zone Ltd.,Services,ADANIPORTS.NS
2,Apollo Hospitals Enterprise Ltd.,Healthcare,APOLLOHOSP.NS
3,Asian Paints Ltd.,Consumer Durables,ASIANPAINT.NS
4,Axis Bank Ltd.,Financial Services,AXISBANK.NS
5,Bajaj Auto Ltd.,Automobile and Auto Components,BAJAJ-AUTO.NS
6,Bajaj Finserv Ltd.,Financial Services,BAJAJFINSV.NS
7,Bajaj Finance Ltd.,Financial Services,BAJFINANCE.NS
8,Bharti Airtel Ltd.,Telecommunication,BHARTIARTL.NS
9,Bharat Petroleum Corporation Ltd.,Oil Gas & Consumable Fuels,BPCL.NS


In [12]:
nse=pd.read_csv("NSE_TICKER_SECTOR_LIST.csv")

# DATABASING NIFTY 

In [13]:
data_list = []

# Loop over each ticker in nifty DataFrame
for (ticker, sector) in zip(nifty["Ticker"], nifty["SECTOR"]):

    # Download historical data for the ticker
    data = yf.download(ticker, start='2017-01-01', end='2023-01-01')

    # Resample the data into 1-month intervals
    monthly_data = data.resample('M').apply(lambda x: x.tolist())


    # Create overlapping two-month intervals
    for i in range(len(monthly_data) - 1):
        two_month_data = monthly_data.iloc[i:i + 2]


        data_list.append({
            "Ticker": ticker,
            "Sector": sector,
            "Date": two_month_data.index[-1],
            "Close": two_month_data['Close'][0] + two_month_data['Close'][1],
            "High": two_month_data['High'][0] + two_month_data['High'][1],
            "Low": two_month_data['Low'][0] + two_month_data['Low'][1],
            "Open": two_month_data['Open'][0] + two_month_data['Open'][1],
            "Volume": two_month_data['Volume'][0] + two_month_data['Volume'][1]
        })

# Convert the list of dictionaries to a DataFrame
database = pd.DataFrame(data_list)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [14]:
database["mean_reversion"]=0
database["high_mean"]=0
database["low_mean"]=0
database["kurto"]=0
database["skew"]=0
database["spectral_entropy"]=0
database["omega"]=0
database["alpha"]=0
database["beta"]=0
database["ADX_SCORE"]=0
database["Mean/Std"]=0
database["Mean/Var"]=0
database["Mean/Median"]=0
database["SLOPE"]=0
database['deviation'] = 0
database["serial_no"]=0
database["bollinger_count"]=0
database['ADX'] = 0
omega_weight = 1000


for i in range(len(database)):
   database["serial_no"][i]=i
   database["bollinger_count"][i]=bollinger_cross(database["Close"][i])
   database["mean_reversion"][i]=mean_rev_para(database["Close"][i])
   database["high_mean"][i]=high_mean(database["Close"][i])
   database["low_mean"][i]=low_mean(database["Close"][i])
   database["kurto"][i]=kurto(database["Close"][i])
   database["skew"][i]=skew(database["Close"][i])
   database["spectral_entropy"][i]=spectral_entropy(database["Close"][i])
   database["omega"][i],database["alpha"][i],database["beta"][i]=garch(database["Close"][i])
   database["ADX_SCORE"][i]=ADX_SCORE(database["High"][i],database["Low"][i],database["Close"][i])
   database["Mean/Std"][i],database["Mean/Var"][i],database["Mean/Median"][i]=ts_charactertsics(database["Close"][i])
   database["SLOPE"][i]=slope(database["Close"][i])
   database["deviation"][i]=shoulder_distance(database["Close"][i])
   database["ADX"][i]=adx(database["Close"][i])











C:\Users\harsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\arch\univariate\base.py:766: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
C:\Users\harsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\arch\univariate\base.py:766: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
C:\Users\harsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\arch\univariate\base.py:766: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmi

In [15]:
# databasing=pd.read_csv("search_database.csv")

In [16]:
# databasing["Close"]=databasing["Close"].str.replace(",","").astype(float)
# databasing["High"]=databasing["High"].str.replace(",","").astype(float)
# databasing["Low"]=databasing["Low"].str.replace(",","").astype(float)
# databasing["Open"]=databasing["Open"].str.replace(",","").astype(float)
# databasing["Volume"]=databasing["Volume"].str.replace(",","").astype(float)


In [17]:
# database=pd.read_csv("search_database.csv")

In [18]:
database["vector"]=0
vector_columns = ["mean_reversion", "high_mean", "low_mean", "kurto", "skew",
                   "spectral_entropy", "omega", "alpha", "beta", "ADX_SCORE", "SLOPE","bollinger_count", "ADX"]

In [19]:
database.sort_values(by="Date",inplace=True)
database.reset_index(drop=True,inplace=True)
database["vector"]=np.array(database["vector"])
x=database[vector_columns].values
database["vector"]=x.tolist()


In [20]:
database

,Ticker,Sector,Date,Close,High,Low,Open,Volume,mean_reversion,high_mean,...,ADX_SCORE,Mean/Std,Mean/Var,Mean/Median,SLOPE,deviation,serial_no,bollinger_count,ADX,vector
0,ADANIENT.NS,Metals & Mining,2017-02-28,"[41.93014907836914, 41.712188720703125, 42.393...","[42.393314361572266, 42.17535400390625, 42.938...","[41.603206634521484, 41.30351257324219, 41.848...","[42.393314361572266, 42.01188278198242, 41.957...","[4214451, 2311286, 3999976, 4236903, 2129980, ...",0,0.884239,...,0,11.381806,2.729599,0.959224,0.686552,1,0,2,0,"[0.0, 0.8842385622738317, 1.1377857580524209, ..."
1,ICICIBANK.NS,Financial Services,2017-02-28,"[229.0454559326172, 231.27272033691406, 228.27...","[233.0, 232.18182373046875, 232.5, 235.0, 237....","[225.5454559326172, 226.63636779785156, 227.63...","[233.0, 229.18182373046875, 231.13636779785156...","[10404350, 10236706, 6901181, 13830668, 905514...",0,0.938645,...,0,23.803293,2.287274,1.004012,0.249381,1,1623,0,0,"[0.0, 0.9386453714730948, 1.085180233984027, -..."
2,HINDUNILVR.NS,Fast Moving Consumer Goods,2017-02-28,"[825.3499755859375, 829.8499755859375, 821.599...","[828.0, 832.7999877929688, 837.4000244140625, ...","[819.0999755859375, 819.5, 816.5499877929688, ...","[828.0, 826.4500122070312, 829.8499755859375, ...","[439748, 617599, 1298675, 800848, 545826, 3232...",0,0.976491,...,0,62.694762,4.630996,0.996292,0.127673,5,1552,1,0,"[0.0, 0.9764913025956864, 1.0330650892862052, ..."
3,HINDALCO.NS,Metals & Mining,2017-02-28,"[158.85000610351562, 158.85000610351562, 158.8...","[159.75, 161.0, 161.60000610351562, 162.649993...","[154.1999969482422, 157.0500030517578, 157.449...","[155.85000610351562, 159.85000610351562, 158.8...","[9537127, 6841113, 7732019, 9921362, 6830935, ...",1,0.902929,...,0,16.544705,1.524540,0.985036,0.408199,4,1481,1,1,"[1.0, 0.9029293171628797, 1.1363765837271003, ..."
4,HEROMOTOCO.NS,Automobile and Auto Components,2017-02-28,"[3030.550048828125, 2986.550048828125, 3004.89...","[3068.949951171875, 3016.14990234375, 3017.300...","[2992.550048828125, 2962.550048828125, 2971.0,...","[3042.0, 2978.0, 2999.0, 3034.89990234375, 308...","[213654, 454277, 396816, 383925, 266915, 21162...",0,0.955222,...,0,39.083835,0.486516,1.003369,0.099381,3,1410,2,0,"[0.0, 0.9552218051722206, 1.0513020758185971, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3526,EICHERMOT.NS,Automobile and Auto Components,2022-12-31,"[3798.39990234375, 3729.199951171875, 3675.25,...","[3889.64990234375, 3849.800048828125, 3747.0, ...","[3751.949951171875, 3695.0, 3648.35009765625, ...","[3889.64990234375, 3849.800048828125, 3698.0, ...","[746053, 1177767, 738117, 631943, 505944, 4904...",1,0.897174,...,-1,20.368286,0.121740,1.009023,-0.373783,1,1135,2,1,"[1.0, 0.8971736782345308, 1.094478993680773, -..."
3527,BAJAJ-AUTO.NS,Automobile and Auto Components,2022-12-31,"[3732.699951171875, 3736.800048828125, 3773.5,...","[3744.199951171875, 3746.0, 3814.0, 3809.89990...","[3663.0, 3693.60009765625, 3731.050048828125, ...","[3686.550048828125, 3746.0, 3754.0, 3779.44995...","[514106, 317571, 637882, 230932, 272013, 24943...",0,0.969494,...,0,55.832101,0.852076,1.005949,-0.070806,5,425,3,0,"[0.0, 0.969493811106889, 1.0329168348795679, -..."
3528,DRREDDY.NS,Healthcare,2022-12-31,"[4535.2998046875, 4555.64990234375, 4589.89990...","[4580.0, 4645.0, 4600.0, 4600.0, 4547.29980468...","[4435.25, 4545.0498046875, 4527.35009765625, 4...","[4435.25, 4545.0498046875, 4555.64990234375, 4...","[1070196, 741756, 353215, 492676, 420017, 7131...",0,0.962457,...,0,48.607789,0.534844,1.000845,-0.153936,1,1064,0,0,"[0.0, 0.9624565181902958, 1.0424842851322005, ..."
3529,TECHM.NS,Information Technology,2022-12-31,"[1070.9000244140625, 1082.0, 1052.849975585937...","[1080.0, 1098.699951171875, 1071.1500244140625...","[1047.1500244140625, 1066.3499755859375, 1050....","[1069.0, 1071.0, 1070.0, 1053.0, 1058.94995117...","[4709472, 4104485, 3460133, 2531080, 2588984, ...",0,0.942394,...,0,36.4

In [21]:
def get_your_strategy_database(data):
    
    data["strategy"]=0
    
    # print(data["ADX_SCORE"])
    for i in range(len(data)):
        if ((data['ADX_SCORE'].iloc[i] == 1 and data['SLOPE'].iloc[i] >= 0.5)):
            data["strategy"][i]=1 # 1                                                              ## STRATEGY 1 FOR HIGH TREND UPWARDS
        elif ((data['ADX_SCORE'][i] == -1 and data['SLOPE'][i] <= -0.5)):
            data["strategy"][i]=2  # 2                                                             ##  STRATEGY 2 FOR HIGH TREND DOWNWARDS
        elif ((data['ADX_SCORE'][i] == 1 and(data['SLOPE'][i] > 0 and data['SLOPE'][i] <= 0.5))):
            data["strategy"][i] = 3      # 3                                                           ## ## STRATEGY 3 FOR LOW TREND UPWARDS
        elif ((data['ADX_SCORE'][i] == -1 and(data['SLOPE'][i] < 0 and data['SLOPE'][i] >= -0.5))):
            data["strategy"][i] = 4      # 4                                                           ## STRATEGY 4 FOR LOW TREND DOWNWARDS
        elif (data['mean_reversion'][i] == 1 and data['ADX'][i] == 1 and data["SLOPE"][i] > 0):
            data["strategy"][i] = 5       # 5                                                           ## STRATEGY 5 FOR MEAN REVERSIONS uptrend
        elif (data['mean_reversion'][i] == 1 and data['ADX'][i] == 1 and data["SLOPE"][i] < 0): 
            data["strategy"][i] = 6         # 6                                                         ## STRATEGY 6 GENERAL downtrend
        else: 
            data["strategy"][i] = 7  
        
    return data
        

In [22]:
def get_your_strategy(data):
    
    data["strategy"]=0
    
    # print(data["ADX_SCORE"])
    for i in range(len(data)):
        if ((data['ADX_SCORE'] == 1 and data['SLOPE'] >= 0.5)):
            stratt =  1  # 1                                                              ## STRATEGY 1 FOR HIGH TREND UPWARDS
        elif ((data['ADX_SCORE'] == -1 and data['SLOPE'] <= -0.5)):
            stratt = 2    # 2                                                             ##  STRATEGY 2 FOR HIGH TREND DOWNWARDS
        elif ((data['ADX_SCORE'] == 1 and(data['SLOPE'] > 0 and data['SLOPE'] <= 0.5))):
            stratt = 3      # 3                                                           ## ## STRATEGY 3 FOR LOW TREND UPWARDS
        elif ((data['ADX_SCORE'] == -1 and(data['SLOPE'] < 0 and data['SLOPE'] >= -0.5))):
            stratt = 4      # 4                                                           ## STRATEGY 4 FOR LOW TREND DOWNWARDS
        elif (data['mean_reversion'] == 1 and data['ADX'] == 1 and data["SLOPE"] > 0):
            stratt = 5       # 5                                                           ## STRATEGY 5 FOR MEAN REVERSIONS uptrend
        elif (data['mean_reversion'] == 1 and data['ADX'] == 1 and data["SLOPE"] < 0): 
            stratt = 6         # 6                                                         ## STRATEGY 6 GENERAL downtrend
        else: 
            stratt = 7  
        
    return stratt
        

In [23]:
database = get_your_strategy_database(database)

In [24]:
# database.to_csv("search_database.csv")

In [25]:
database

,Ticker,Sector,Date,Close,High,Low,Open,Volume,mean_reversion,high_mean,...,Mean/Std,Mean/Var,Mean/Median,SLOPE,deviation,serial_no,bollinger_count,ADX,vector,strategy
0,ADANIENT.NS,Metals & Mining,2017-02-28,"[41.93014907836914, 41.712188720703125, 42.393...","[42.393314361572266, 42.17535400390625, 42.938...","[41.603206634521484, 41.30351257324219, 41.848...","[42.393314361572266, 42.01188278198242, 41.957...","[4214451, 2311286, 3999976, 4236903, 2129980, ...",0,0.884239,...,11.381806,2.729599,0.959224,0.686552,1,0,2,0,"[0.0, 0.8842385622738317, 1.1377857580524209, ...",7
1,ICICIBANK.NS,Financial Services,2017-02-28,"[229.0454559326172, 231.27272033691406, 228.27...","[233.0, 232.18182373046875, 232.5, 235.0, 237....","[225.5454559326172, 226.63636779785156, 227.63...","[233.0, 229.18182373046875, 231.13636779785156...","[10404350, 10236706, 6901181, 13830668, 905514...",0,0.938645,...,23.803293,2.287274,1.004012,0.249381,1,1623,0,0,"[0.0, 0.9386453714730948, 1.085180233984027, -...",7
2,HINDUNILVR.NS,Fast Moving Consumer Goods,2017-02-28,"[825.3499755859375, 829.8499755859375, 821.599...","[828.0, 832.7999877929688, 837.4000244140625, ...","[819.0999755859375, 819.5, 816.5499877929688, ...","[828.0, 826.4500122070312, 829.8499755859375, ...","[439748, 617599, 1298675, 800848, 545826, 3232...",0,0.976491,...,62.694762,4.630996,0.996292,0.127673,5,1552,1,0,"[0.0, 0.9764913025956864, 1.0330650892862052, ...",7
3,HINDALCO.NS,Metals & Mining,2017-02-28,"[158.85000610351562, 158.85000610351562, 158.8...","[159.75, 161.0, 161.60000610351562, 162.649993...","[154.1999969482422, 157.0500030517578, 157.449...","[155.85000610351562, 159.85000610351562, 158.8...","[9537127, 6841113, 7732019, 9921362, 6830935, ...",1,0.902929,...,16.544705,1.524540,0.985036,0.408199,4,1481,1,1,"[1.0, 0.9029293171628797, 1.1363765837271003, ...",5
4,HEROMOTOCO.NS,Automobile and Auto Components,2017-02-28,"[3030.550048828125, 2986.550048828125, 3004.89...","[3068.949951171875, 3016.14990234375, 3017.300...","[2992.550048828125, 2962.550048828125, 2971.0,...","[3042.0, 2978.0, 2999.0, 3034.89990234375, 308...","[213654, 454277, 396816, 383925, 266915, 21162...",0,0.955222,...,39.083835,0.486516,1.003369,0.099381,3,1410,2,0,"[0.0, 0.9552218051722206, 1.0513020758185971, ...",7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3526,EICHERMOT.NS,Automobile and Auto Components,2022-12-31,"[3798.39990234375, 3729.199951171875, 3675.25,...","[3889.64990234375, 3849.800048828125, 3747.0, ...","[3751.949951171875, 3695.0, 3648.35009765625, ...","[3889.64990234375, 3849.800048828125, 3698.0, ...","[746053, 1177767, 738117, 631943, 505944, 4904...",1,0.897174,...,20.368286,0.121740,1.009023,-0.373783,1,1135,2,1,"[1.0, 0.8971736782345308, 1.094478993680773, -...",4
3527,BAJAJ-AUTO.NS,Automobile and Auto Components,2022-12-31,"[3732.699951171875, 3736.800048828125, 3773.5,...","[3744.199951171875, 3746.0, 3814.0, 3809.89990...","[3663.0, 3693.60009765625, 3731.050048828125, ...","[3686.550048828125, 3746.0, 3754.0, 3779.44995...","[514106, 317571, 637882, 230932, 272013, 24943...",0,0.969494,...,55.832101,0.852076,1.005949,-0.070806,5,425,3,0,"[0.0, 0.969493811106889, 1.0329168348795679, -...",7
3528,DRREDDY.NS,Healthcare,2022-12-31,"[4535.2998046875, 4555.64990234375, 4589.89990...","[4580.0, 4645.0, 4600.0, 4600.0, 4547.29980468...","[4435.25, 4545.0498046875, 4527.35009765625, 4...","[4435.25, 4545.0498046875, 4555.64990234375, 4...","[1070196, 741756, 353215, 492676, 420017, 7131...",0,0.962457,...,48.607789,0.534844,1.000845,-0.153936,1,1064,0,0,"[0.0, 0.9624565181902958, 1.0424842851322005, ...",7
3529,TECHM.NS,Information Technology,2022-12-31,"[1070.9000244140625, 1082.0, 1052.849975585937...","[1080.0, 1098.699951171875, 1071.1500244140625...","[1047.1500244140625, 1066.3499755859375, 1050....","[1069.0, 1071.0, 1070.0, 1053.0, 1058.94995117...","[4709472, 4104485, 3460133, 2531080, 2588984, ...",0,0.942394,...,36.42186

In [26]:
nse

,Unnamed: 0,SYMBOL,NAME OF COMPANY,Industry,SECTOR
0,0,20MICRONS.NS,20 Microns Limited,Others,Others
1,1,21STCENMGM.NS,21st Century Management Services Limited,Others,Others
2,2,360ONE.NS,360 ONE WAM LIMITED,Financial Services,Financial Services
3,3,3IINFOLTD.NS,3i Infotech Limited,Others,Others
4,4,3MINDIA.NS,3M India Limited,Diversified,Diversified
...,...,...,...,...,...
1998,1998,ZOTA.NS,Zota Health Care LImited,Others,Others
1999,1999,ZUARI.NS,Zuari Agro Chemicals Limited,Others,Others
2000,2000,ZUARIIND.NS,ZUARI INDUSTRIES LIMITED,Others,Others
2001,2001,ZYDUSLIFE.NS,Zydus Lifesciences Limited,Healthcare,Healthcare


# testing mapping 

In [27]:
# start_date="2022-01-01"
# end_date="2022-06-01"
# ticker="AXISBANK.NS"
# # data_of_maps, stocks_of_maps =check(ticker,subtract_two_months(start_date,0),subtract_two_months(end_date,0))
# # mapping =check(ticker,subtract_two_months(start_date,0),subtract_two_months(end_date,0),0.75)
# # mapping =check(ticker,subtract_two_months(start_date,0),subtract_two_months(end_date,0),0.75)
# mapping = check(ticker,start_date,end_date)



In [28]:
def check(ticker, start, end,threshold_sector=0.7):
    
    data = yf.download(ticker, start, end)
    data_reset = data.reset_index()
    
    sector=nse[nse["SYMBOL"]==ticker]["Industry"].values[0]
    

    # Ensure the Date column is a datetime type
    data_reset['Date'] = pd.to_datetime(data_reset['Date'])

    # Create overlapping 2-month periods
    dates_of_maps = []
    stocks_of_maps = []
    seria_no=[]

    # Define the start and end dates for the first chunk
    current_start = pd.to_datetime(start)
    current_end = current_start + pd.DateOffset(months=2)

    while current_end <= pd.to_datetime(end):

        # Filter data for the current 2-month period
        period_data = data_reset[(data_reset['Date'] >= current_start) & (data_reset['Date'] < current_end)]

        if not period_data.empty:
            chunk_data = {
                'sector':sector,
                'ticker':ticker,
                'Close': period_data['Close'].to_numpy(),
                'Open': period_data['Open'].to_numpy(),
                'High': period_data['High'].to_numpy(),
                'Low': period_data['Low'].to_numpy(),
                'Date': period_data['Date'].to_numpy()
            }
            # return(chunk_data)
            # Calculate features
            chunk_data["bollinger_count"]=bollinger_cross(chunk_data["Close"])
            chunk_data["mean_reversion"] = mean_rev_para(chunk_data["Close"])
            chunk_data["high_mean"] = high_mean(chunk_data["Close"])
            chunk_data["low_mean"] = low_mean(chunk_data["Close"])
            chunk_data["kurto"] = kurto(chunk_data["Close"])
            chunk_data["skew"] = skew(chunk_data["Close"])
            chunk_data["spectral_entropy"] = spectral_entropy(chunk_data["Close"])
            chunk_data["omega"], chunk_data["alpha"], chunk_data["beta"] = garch(chunk_data["Close"])
            chunk_data["omega"] *= 1000
            chunk_data["ADX_SCORE"] = ADX_SCORE(chunk_data["High"], chunk_data["Low"], chunk_data["Close"])
            chunk_data["Mean/Std"], chunk_data["Mean/Var"], chunk_data["Mean/Median"] = ts_charactertsics(chunk_data["Close"])
            chunk_data["SLOPE"] = slope(chunk_data["Close"])
            chunk_data['deviation'] = shoulder_distance(chunk_data['Close'])
            chunk_data['ADX'] = adx(chunk_data['Close'])
            
            
            
            x = np.array([chunk_data[col] for col in vector_columns])

            # Define the period for short-term comparison
            start_date = chunk_data['Date'][0]
            end_date = chunk_data['Date'][-1]  # End of the 2-month period

            # Adjust date ranges for matching
            short_term = database[(database["Date"] < start_date) ]
            if 'vector' in short_term.columns:
                short_term['y'] = short_term['vector'].apply(lambda v: np.linalg.norm(np.array(v) - x))
                short_term_overall=short_term.sort_values(by="y", inplace=False)
                short_term_sector=short_term[short_term["Sector"]==sector].sort_values(by="y", inplace=False)
                short_term.sort_values(by="y", inplace=True)


                # Plotting
                
                
                

                plt.title(f"Current Chunk: {chunk_data['Date'][0]}, {ticker}")
                plt.plot(chunk_data["Close"])
                plt.show()

                # Collect dates and tickers
                if (short_term_overall["y"].iloc[0]/short_term_sector["y"].iloc[0])>=threshold_sector:
                    plt.title(f"Sector Match: {short_term_sector['Date'].iloc[0]}, {short_term_sector['Ticker'].iloc[0]}, {short_term_sector['Ticker'].iloc[0]}")
                    plt.plot(short_term_sector["Close"].iloc[0])
                    plt.show()
                    # dates_of_maps.append(short_term_sector['Date'].iloc[0])
                    # stocks_of_maps.append(short_term_sector["Ticker"].iloc[0])
                    seria_no.append(short_term_sector["serial_no"].iloc[0]+1)

                else:
                    plt.title(f"Overall Match: {short_term_overall['Date'].iloc[0]}, {short_term_overall['Ticker'].iloc[0]}, {short_term_overall['Ticker'].iloc[0]}")
                    plt.plot(short_term_overall["Close"].iloc[0])
                    plt.show()
                    # dates_of_maps.append(short_term_overall['Date'].iloc[0])
                    # stocks_of_maps.append(short_term_overall["Ticker"].iloc[0])
                    seria_no.append(short_term_overall["serial_no"].iloc[0]+1)


        # Move to the next chunk (overlapping by 1 month)
        current_start = current_start + pd.DateOffset(months=1)
        current_end = current_start + pd.DateOffset(months=2)

    return dates_of_maps, stocks_of_maps,seria_no

In [56]:
def give_parameters(data):
    data["bollinger_count"]=bollinger_cross(data["Close"])
    data["mean_reversion"] = mean_rev_para(data["Close"])
    data["high_mean"] = high_mean(data["Close"])
    data["low_mean"] = low_mean(data["Close"])
    data["kurto"] = kurto(data["Close"])
    data["skew"] = skew(data["Close"])
    data["spectral_entropy"] = spectral_entropy(data["Close"])
    data["omega"], data["alpha"], data["beta"] = garch(data["Close"])
    data["omega"] *= 1000
    data["ADX_SCORE"] = ADX_SCORE(data["High"], data["Low"], data["Close"])
    data["Mean/Std"], data["Mean/Var"], data["Mean/Median"] = ts_charactertsics(data["Close"])
    data["SLOPE"] = slope(data["Close"])
    data['deviation'] = shoulder_distance(data['Close'])
    data["ADX"]=adx(data["Close"])
    return data
     
    

In [30]:
def mapping(data,ticker):
    data["Date"]=pd.to_datetime(data.index)   
    sector=nse[nse["SYMBOL"]==ticker]["Industry"].values[0]
    chunk_data = {
    'sector':sector,
    'ticker':ticker,
    'Close': data['Close'].to_numpy(),
    'Open': data['Open'].to_numpy(),
    'High': data['High'].to_numpy(),
    'Low': data['Low'].to_numpy(),
    'Date': data['Date'].to_numpy()
    }
    chunk_data=give_parameters(chunk_data)
    x = np.array([chunk_data[col] for col in vector_columns])
    
    short_term = database[(database["Date"] < (chunk_data['Date'][0])) ]
    if 'vector' in short_term.columns:
        short_term['y'] = short_term['vector'].apply(lambda v: np.linalg.norm(np.array(v) - x))
        short_term_overall=short_term.sort_values(by="y", inplace=False)
        short_term_sector=short_term[short_term["Sector"]==sector].sort_values(by="y", inplace=False)
        short_term.sort_values(by="y", inplace=True)
        # print(short_term_overall)
        if (short_term_overall["y"].iloc[0]/short_term_sector["y"].iloc[0])>=0.75:
            return short_term_sector["Date"].iloc[0],short_term_sector["Ticker"].iloc[0]
        else:
            return short_term_overall["Date"][0],short_term_overall["Ticker"][0]
    

In [31]:
def strat(map):
    return database["startegy"].iloc[map]

# Strategy

In [32]:
def strat1(data,n=7):

    
    
    data = data.copy()

    # Calculate True Range (TR)
    data['TR'] = np.maximum(data['High'] - data['Low'], np.maximum(abs(data['High'] - data['Close'].shift(1)), abs(data['Low'] - data['Close'].shift(1))))

    # Calculate directional movement
    data['+DM'] = np.where((data['High'] - data['High'].shift(1)) > (data['Low'].shift(1) - data['Low']),
                         np.maximum(data['High'] - data['High'].shift(1), 0), 0)
    data['-DM'] = np.where((data['Low'].shift(1) - data['Low']) > (data['High'] - data['High'].shift(1)),
                         np.maximum(data['Low'].shift(1) - data['Low'], 0), 0)

    # Smooth the TR, +DM, and -DM values
    data['TR_smooth'] = data['TR'].rolling(window=n).sum()
    data['+DM_smooth'] = data['+DM'].rolling(window=n).sum()
    data['-DM_smooth'] = data['-DM'].rolling(window=n).sum()

    # Calculate +DI and -DI
    data['+DI'] = 100 * (data['+DM_smooth'] / data['TR_smooth'])
    data['-DI'] = 100 * (data['-DM_smooth'] / data['TR_smooth'])

    # Calculate the DX (Directional Movement Index)
    data['DX'] = 100 * (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI']))

    # Calculate the ADX (Average Directional Index)
    data['ADX'] = data['DX'].rolling(window=n).mean()

    # Drop intermediate columns
    data = data.drop(['TR', '+DM', '-DM', 'TR_smooth', '+DM_smooth', '-DM_smooth', 'DX'], axis=1)
    
    
    
    OBV = []
    OBV.append(0)

#Loop through the data set (close price) from the second row (index 1) to the end of the data set
    for i in range(1, len(data)):
     if data['Close'].iloc[i] > data['Close'].iloc[i-1]:
       OBV.append(OBV[-1] + data['Volume'].iloc[i])
     elif data['Close'].iloc[i] < data['Close'].iloc[i-1]:
       OBV.append(OBV[-1] - data['Volume'].iloc[i])
     else:
       OBV.append(OBV[-1])
    data['OBV'] = OBV
    data['OBV_M']=data['OBV'].ewm(span=20,adjust=False).mean()
     # data['OBV_M']=data['OBV'].rolling(window=10).mean()
    
    

    ADX_Signal=[0]
    for i in range(1,len(data)):
       if  data['OBV'].iloc[i]>data['OBV_M'].iloc[i-1] or data['Close'].iloc[i]>data['Close'].iloc[i-1]:
        ADX_Signal.append(1)
       elif  data['OBV'].iloc[i]<data['OBV_M'].iloc[i-1] or  data['Close'].iloc[i]<data['Close'].iloc[i-1]:
        ADX_Signal.append(-1)
       else:
        ADX_Signal.append(0)
    return ADX_Signal


In [33]:
def strat2(data,n=7):

    
    
    data = data.copy()

    # Calculate True Range (TR)
    data['TR'] = np.maximum(data['High'] - data['Low'], np.maximum(abs(data['High'] - data['Close'].shift(1)), abs(data['Low'] - data['Close'].shift(1))))

    # Calculate directional movement
    data['+DM'] = np.where((data['High'] - data['High'].shift(1)) > (data['Low'].shift(1) - data['Low']),
                         np.maximum(data['High'] - data['High'].shift(1), 0), 0)
    data['-DM'] = np.where((data['Low'].shift(1) - data['Low']) > (data['High'] - data['High'].shift(1)),
                         np.maximum(data['Low'].shift(1) - data['Low'], 0), 0)

    # Smooth the TR, +DM, and -DM values
    data['TR_smooth'] = data['TR'].rolling(window=n).sum()
    data['+DM_smooth'] = data['+DM'].rolling(window=n).sum()
    data['-DM_smooth'] = data['-DM'].rolling(window=n).sum()

    # Calculate +DI and -DI
    data['+DI'] = 100 * (data['+DM_smooth'] / data['TR_smooth'])
    data['-DI'] = 100 * (data['-DM_smooth'] / data['TR_smooth'])

    # Calculate the DX (Directional Movement Index)
    data['DX'] = 100 * (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI']))

    # Calculate the ADX (Average Directional Index)
    data['ADX'] = data['DX'].rolling(window=n).mean()

    # Drop intermediate columns
    data = data.drop(['TR', '+DM', '-DM', 'TR_smooth', '+DM_smooth', '-DM_smooth', 'DX'], axis=1)
    
    
    
    OBV = []
    OBV.append(0)

#Loop through the data set (close price) from the second row (index 1) to the end of the data set
    for i in range(1, len(data)):
     if data['Close'].iloc[i] > data['Close'].iloc[i-1]:
       OBV.append(OBV[-1] + data['Volume'].iloc[i])
     elif data['Close'].iloc[i] < data['Close'].iloc[i-1]:
       OBV.append(OBV[-1] - data['Volume'].iloc[i])
     else:
       OBV.append(OBV[-1])
    data['OBV'] = OBV
    data['OBV_M']=data['OBV'].ewm(span=20,adjust=False).mean()
     # data['OBV_M']=data['OBV'].rolling(window=10).mean()
    
    

    ADX_Signal=[0]
    for i in range(1,len(data)):
       if  data['OBV'].iloc[i]>data['OBV_M'].iloc[i-1] or data['Close'].iloc[i]>data['Close'].iloc[i-1]:
        ADX_Signal.append(1)
       elif  data['OBV'].iloc[i]<data['OBV_M'].iloc[i-1] or  data['Close'].iloc[i]<data['Close'].iloc[i-1]:
        ADX_Signal.append(-1)
       else:
        ADX_Signal.append(0)
    return ADX_Signal


In [34]:
def strat3(data,n=7):

    
    
    data = data.copy()

    # Calculate True Range (TR)
    data['TR'] = np.maximum(data['High'] - data['Low'], np.maximum(abs(data['High'] - data['Close'].shift(1)), abs(data['Low'] - data['Close'].shift(1))))

    # Calculate directional movement
    data['+DM'] = np.where((data['High'] - data['High'].shift(1)) > (data['Low'].shift(1) - data['Low']),
                         np.maximum(data['High'] - data['High'].shift(1), 0), 0)
    data['-DM'] = np.where((data['Low'].shift(1) - data['Low']) > (data['High'] - data['High'].shift(1)),
                         np.maximum(data['Low'].shift(1) - data['Low'], 0), 0)

    # Smooth the TR, +DM, and -DM values
    data['TR_smooth'] = data['TR'].rolling(window=n).sum()
    data['+DM_smooth'] = data['+DM'].rolling(window=n).sum()
    data['-DM_smooth'] = data['-DM'].rolling(window=n).sum()

    # Calculate +DI and -DI
    data['+DI'] = 100 * (data['+DM_smooth'] / data['TR_smooth'])
    data['-DI'] = 100 * (data['-DM_smooth'] / data['TR_smooth'])

    # Calculate the DX (Directional Movement Index)
    data['DX'] = 100 * (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI']))

    # Calculate the ADX (Average Directional Index)
    data['ADX'] = data['DX'].rolling(window=n).mean()

    # Drop intermediate columns
    data = data.drop(['TR', '+DM', '-DM', 'TR_smooth', '+DM_smooth', '-DM_smooth', 'DX'], axis=1)
    
    
    
    OBV = []
    OBV.append(0)

#Loop through the data set (close price) from the second row (index 1) to the end of the data set
    for i in range(1, len(data)):
     if data['Close'].iloc[i] > data['Close'].iloc[i-1]:
       OBV.append(OBV[-1] + data['Volume'].iloc[i])
     elif data['Close'].iloc[i] < data['Close'].iloc[i-1]:
       OBV.append(OBV[-1] - data['Volume'].iloc[i])
     else:
       OBV.append(OBV[-1])
    data['OBV'] = OBV
    data['OBV_M']=data['OBV'].ewm(span=20,adjust=False).mean()
     # data['OBV_M']=data['OBV'].rolling(window=10).mean()
    
    

    ADX_Signal=[0]
    for i in range(1,len(data)):
       if  data['OBV'].iloc[i]>data['OBV_M'].iloc[i-1] or data['Close'].iloc[i]>data['Close'].iloc[i-1]:
        ADX_Signal.append(1)
       elif  data['OBV'].iloc[i]<data['OBV_M'].iloc[i-1] or  data['Close'].iloc[i]<data['Close'].iloc[i-1]:
        ADX_Signal.append(-1)
       else:
        ADX_Signal.append(0)
    return ADX_Signal


In [35]:
def strat4(data,n=7):

    
    
    data = data.copy()

    # Calculate True Range (TR)
    data['TR'] = np.maximum(data['High'] - data['Low'], np.maximum(abs(data['High'] - data['Close'].shift(1)), abs(data['Low'] - data['Close'].shift(1))))

    # Calculate directional movement
    data['+DM'] = np.where((data['High'] - data['High'].shift(1)) > (data['Low'].shift(1) - data['Low']),
                         np.maximum(data['High'] - data['High'].shift(1), 0), 0)
    data['-DM'] = np.where((data['Low'].shift(1) - data['Low']) > (data['High'] - data['High'].shift(1)),
                         np.maximum(data['Low'].shift(1) - data['Low'], 0), 0)

    # Smooth the TR, +DM, and -DM values
    data['TR_smooth'] = data['TR'].rolling(window=n).sum()
    data['+DM_smooth'] = data['+DM'].rolling(window=n).sum()
    data['-DM_smooth'] = data['-DM'].rolling(window=n).sum()

    # Calculate +DI and -DI
    data['+DI'] = 100 * (data['+DM_smooth'] / data['TR_smooth'])
    data['-DI'] = 100 * (data['-DM_smooth'] / data['TR_smooth'])

    # Calculate the DX (Directional Movement Index)
    data['DX'] = 100 * (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI']))

    # Calculate the ADX (Average Directional Index)
    data['ADX'] = data['DX'].rolling(window=n).mean()

    # Drop intermediate columns
    data = data.drop(['TR', '+DM', '-DM', 'TR_smooth', '+DM_smooth', '-DM_smooth', 'DX'], axis=1)
    
    
    
    OBV = []
    OBV.append(0)

#Loop through the data set (close price) from the second row (index 1) to the end of the data set
    for i in range(1, len(data)):
     if data['Close'].iloc[i] > data['Close'].iloc[i-1]:
       OBV.append(OBV[-1] + data['Volume'].iloc[i])
     elif data['Close'].iloc[i] < data['Close'].iloc[i-1]:
       OBV.append(OBV[-1] - data['Volume'].iloc[i])
     else:
       OBV.append(OBV[-1])
    data['OBV'] = OBV
    data['OBV_M']=data['OBV'].ewm(span=20,adjust=False).mean()
     # data['OBV_M']=data['OBV'].rolling(window=10).mean()
    
    

    ADX_Signal=[0]
    for i in range(1,len(data)):
       if  data['OBV'].iloc[i]>data['OBV_M'].iloc[i-1] or data['Close'].iloc[i]>data['Close'].iloc[i-1]:
        ADX_Signal.append(1)
       elif  data['OBV'].iloc[i]<data['OBV_M'].iloc[i-1] or  data['Close'].iloc[i]<data['Close'].iloc[i-1]:
        ADX_Signal.append(-1)
       else:
        ADX_Signal.append(0)
    return ADX_Signal


In [36]:
def strat5(data):                         ### FINAL FOR MEAN REVERSION UPWARDS :))
    signal = [0]
    min_price = data['Close'][0]
    max_price = data['Close'][0]
    for i in range(1,len(data)):
        price = data['Close'][i]
        if(price  > max_price):
            max_price = price
        elif(price < min_price):
            min_price = price
        lvl3 = max_price + 0.01*(max_price - min_price)
        lvl4 = max_price - 0.65*(max_price - min_price)
        
        if(price <= lvl4):
            signal.append(1)
        elif(price >= lvl3):
            signal.append(-1)
        else:
            signal.append(0)
    data['signals'] = signal
    return data["signals"]

In [37]:
def strat6(data,n=7):

    
    
    data = data.copy()

    # Calculate True Range (TR)
    data['TR'] = np.maximum(data['High'] - data['Low'], np.maximum(abs(data['High'] - data['Close'].shift(1)), abs(data['Low'] - data['Close'].shift(1))))

    # Calculate directional movement
    data['+DM'] = np.where((data['High'] - data['High'].shift(1)) > (data['Low'].shift(1) - data['Low']),
                         np.maximum(data['High'] - data['High'].shift(1), 0), 0)
    data['-DM'] = np.where((data['Low'].shift(1) - data['Low']) > (data['High'] - data['High'].shift(1)),
                         np.maximum(data['Low'].shift(1) - data['Low'], 0), 0)

    # Smooth the TR, +DM, and -DM values
    data['TR_smooth'] = data['TR'].rolling(window=n).sum()
    data['+DM_smooth'] = data['+DM'].rolling(window=n).sum()
    data['-DM_smooth'] = data['-DM'].rolling(window=n).sum()

    # Calculate +DI and -DI
    data['+DI'] = 100 * (data['+DM_smooth'] / data['TR_smooth'])
    data['-DI'] = 100 * (data['-DM_smooth'] / data['TR_smooth'])

    # Calculate the DX (Directional Movement Index)
    data['DX'] = 100 * (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI']))

    # Calculate the ADX (Average Directional Index)
    data['ADX'] = data['DX'].rolling(window=n).mean()

    # Drop intermediate columns
    data = data.drop(['TR', '+DM', '-DM', 'TR_smooth', '+DM_smooth', '-DM_smooth', 'DX'], axis=1)
    
    
    
    OBV = []
    OBV.append(0)

#Loop through the data set (close price) from the second row (index 1) to the end of the data set
    for i in range(1, len(data)):
     if data['Close'].iloc[i] > data['Close'].iloc[i-1]:
       OBV.append(OBV[-1] + data['Volume'].iloc[i])
     elif data['Close'].iloc[i] < data['Close'].iloc[i-1]:
       OBV.append(OBV[-1] - data['Volume'].iloc[i])
     else:
       OBV.append(OBV[-1])
    data['OBV'] = OBV
    data['OBV_M']=data['OBV'].ewm(span=20,adjust=False).mean()
     # data['OBV_M']=data['OBV'].rolling(window=10).mean()
    
    

    ADX_Signal=[0]
    for i in range(1,len(data)):
       if  data['OBV'].iloc[i]>data['OBV_M'].iloc[i-1] or data['Close'].iloc[i]>data['Close'].iloc[i-1]:
        ADX_Signal.append(1)
       elif  data['OBV'].iloc[i]<data['OBV_M'].iloc[i-1] or  data['Close'].iloc[i]<data['Close'].iloc[i-1]:
        ADX_Signal.append(-1)
       else:
        ADX_Signal.append(0)
    return ADX_Signal


# FINAL SIGNALS

In [38]:
def signal_generator(data,stratt):
    if stratt==1:
            return strat1(data)
    elif stratt==2:
            return strat2(data)
    elif stratt==3:
            return strat3(data)
    elif stratt==4:
            return strat4(data)
    elif stratt==5:
            return strat5(data)
    else:
            return strat6(data)
   
    
        

In [39]:
def get_your_signals(ticker,start_date,end_date,refresh_rate=15):
    final_signals=[]
    strat_used=[]
    x=30-refresh_rate
    stock=yf.download(ticker,start_date,end_date)
    stock["datetime"]=stock.index
    start_date=pd.to_datetime(start_date)
    end_date=pd.to_datetime(end_date)
    end_date=start_date
    start=start_date
    i=0
    flag=0
    while i < len(stock):
        if(start>stock.index[-1] or flag==1):
            break
        end=start
        start=start+pd.DateOffset(months=-2)
        # print(start)
        data=yf.download(ticker,start,end)
        copy_data=data.copy()
    
       
       
        
        mapp,tick= mapping(data,ticker)
        print(mapp,tick)
        start_date = (mapp + relativedelta(days=-refresh_rate)).strftime('%Y-%m-%d')
        end_date = (mapp + relativedelta(days=+x)).strftime('%Y-%m-%d')
        data = yf.download(tick, start_date, end_date)
        data["Date"]=data.index
        data = {
    'sector':sector,
    'ticker':ticker,
    'Close': data['Close'].to_numpy(),
    'Open': data['Open'].to_numpy(),
    'High': data['High'].to_numpy(),
    'Low': data['Low'].to_numpy(),
    'Date': data['Date'].to_numpy()
        }
        data = give_parameters(data)
        # print(data)
        stratt=get_your_strategy(data)

        strat=stratt
        # print(stratt)

        if i+refresh_rate>len(stock):
            signals=signal_generator(stock[:][i:],stratt)
            flag=1
        else:
            signals=signal_generator(stock[:][i:i+refresh_rate],stratt)
        if not isinstance(signals, list):
            signals=signals.tolist()
        final_signals.extend(signals)
        if i+refresh_rate<len(stock):
            start=stock["datetime"][i+refresh_rate]  
            # print(start)         
        i=i+refresh_rate
        print(i)
        strat_used.extend([strat]*refresh_rate)
    while len(strat_used)>len(stock):
        strat_used.pop()
    return stock,final_signals,strat_used
    
    




        
    

# Backtesting 

In [40]:
# class Tradingstragegy:

#   def __init__(self,df):
#     self.df=df
#     self.capital=1000000
#     self.current_position=0
#     self.current_quantity=0
#     self.current_enter_portfoliovalue=1000000
#     self.portfolio_value=[]
#     self.position=[]

#     self.current_portfolio_value=1000000
#     self.current_maximum_portfolio_value=1000000
#     self.current_minimum_portfolio_value=1000000

#     self.current_maximum_drawdown=0
#     self.drawdown=[]
#     self.current_maximum_dip=0
#     self.dip=[]
#     self.close=df['Close']
#     self.signal=df['signals']

#     self.entry=[]
#     self.exit=[]
#     self.quantity_tradwise=[]
#     self.tradetype=[]
#     self.returns=[]

#     self.new_signals=[]


#     self.longstoploss=0.05
#     self.longtakeprofit=.05
#     self.longtrallingstoploss=0.05
#     self.longtrallingtakeprofit=.05
#     self.longatr=2

#     self.shortstoploss=.05
#     self.shorttakeprofit=0.05
#     self.shorttrallingstoploss=.05
#     self.shorttrallingtakeprofit=0.05
#     self.shortatr=2






#   def calculate_atr(self, period=14):
#       """
#       Calculate the Average True Range (ATR) for the given data.

#       Parameters:
#       df (DataFrame): DataFrame with columns 'High', 'Low', and 'Close'.
#       period (int): The number of periods to use for the ATR calculation (default is 14).

#       Returns:
#       DataFrame: The input DataFrame with an additional 'ATR' column.
#       """
#       self.df['High-Low'] = self.df['High'] - self.df['Low']
#       self.df['High-PrevClose'] = abs(self.df['High'] - self.df['Close'].shift(1))
#       self.df['Low-PrevClose'] = abs(self.df['Low'] - self.df['Close'].shift(1))
#       close_series = pd.Series(self.close)

#       # Calculate the 3-day and 22-day rolling averages
#       self.roll3 = close_series.rolling(window=3).mean()
#       self.roll22 = close_series.rolling(window=22).mean()

#       self.df['TrueRange'] = self.df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
#       self.df['ATR'] = self.df['TrueRange'].rolling(window=period).mean()

#       self.df.drop(columns=['High-Low', 'High-PrevClose', 'Low-PrevClose', 'TrueRange'], inplace=True)





#   def enter_long(self,index):
#     self.current_position=1
#     self.current_quantity=self.capital//self.close[index]
#     self.capital=self.capital%self.close[index]
#     self.current_maximum_dip=0
#     self.current_maximum_drawdown=0
#     self.current_maximum_portfolio_value=self.current_portfolio_value
#     self.current_minimum_portfolio_value=self.current_portfolio_value

#     self.entry.append(index)
#     self.tradetype.append('Long')
#     self.new_signals.append(1)
#     self.portfolio_value.append(self.current_portfolio_value)
#     self.current_enter_portfoliovalue=self.current_portfolio_value
#     self.position.append(self.current_position)
#     self.quantity_tradwise.append(self.current_quantity)

#   def enter_short(self,index):
#     self.current_position=-1
#     self.current_quantity=self.capital//self.close[index]
#     self.capital=self.capital+self.current_quantity*self.close[index]
#     self.current_maximum_dip=0
#     self.current_maximum_drawdown=0
#     self.current_maximum_portfolio_value=self.current_portfolio_value
#     self.current_minimum_portfolio_value=self.current_portfolio_value

#     self.entry.append(index)
#     self.tradetype.append('Short')
#     self.new_signals.append(-1)
#     self.portfolio_value.append(self.current_portfolio_value)
#     self.current_enter_portfoliovalue=self.current_portfolio_value
#     self.position.append(self.current_position)
#     self.quantity_tradwise.append(self.current_quantity)


#   def exit_long(self,index):
#     self.current_position=0
#     self.capital=self.capital+self.current_quantity*self.close[index]
#     self.current_portfolio_value=self.capital
#     self.drawdown.append(self.current_maximum_drawdown)
#     self.dip.append(self.current_maximum_dip)


#     self.returns.append((self.close[index]-self.close[self.entry[-1]])*100/self.close[index])
#     self.exit.append(index)
#     self.portfolio_value.append(self.current_portfolio_value)
#     self.position.append(self.current_position)
#     self.new_signals.append(-1)

#   def exit_short(self,index):
#     self.current_position=0
#     self.capital=self.capital-self.current_quantity*self.close[index]
#     self.current_portfolio_value=self.capital
#     self.drawdown.append(self.current_maximum_drawdown)
#     self.dip.append(self.current_maximum_dip)

#     self.returns.append((self.close[self.entry[-1]]-self.close[index])*100/self.close[self.entry[-1]])
#     self.exit.append(index)
#     self.portfolio_value.append(self.current_portfolio_value)
#     self.position.append(self.current_position)
#     self.new_signals.append(1)


#   def current_drawdown(self):
#     return (self.current_maximum_portfolio_value-self.current_portfolio_value)*100/self.current_portfolio_value

#   def current_dip(self):
#     return (self.current_enter_portfoliovalue-self.current_portfolio_value)*100/self.current_portfolio_value


#   def update_noposition(self,index):
#     self.current_portfolio_value=self.capital
#     self.portfolio_value.append(self.current_portfolio_value)
#     self.position.append(self.current_position)
#     self.new_signals.append(0)

#   def update_longposition(self,index):
#     self.current_portfolio_value=self.capital+self.current_quantity*self.close[index]
#     self.portfolio_value.append(self.current_portfolio_value)
#     self.current_maximum_portfolio_value=max(self.current_portfolio_value,self.current_maximum_portfolio_value)
#     self.current_minimum_portfolio_value=min(self.current_portfolio_value,self.current_minimum_portfolio_value)
#     self.current_maximum_drawdown=max(self.current_maximum_drawdown,self.current_drawdown())
#     self.current_maximum_dip=max(self.current_maximum_dip,self.current_dip())
#     self.new_signals.append(0)


#   def update_shortposition(self,index):
#     self.current_portfolio_value=self.capital-self.current_quantity*self.close[index]
#     self.portfolio_value.append(self.current_portfolio_value)
#     self.current_maximum_portfolio_value=max(self.current_portfolio_value,self.current_maximum_portfolio_value)
#     self.current_minimum_portfolio_value=min(self.current_portfolio_value,self.current_minimum_portfolio_value)
#     self.current_maximum_drawdown=max(self.current_maximum_drawdown,self.current_drawdown())
#     self.current_maximum_dip=max(self.current_maximum_dip,self.current_dip())
#     self.new_signals.append(0)

#   def check_long_exitcondition(self,index):
#     self.current_portfolio_value=self.capital+self.current_quantity*self.close[index]

#     if(self.current_portfolio_value<self.current_enter_portfoliovalue*(1-self.longstoploss)):

#       self.exit_long(index)
#       return 1
#     elif(self.current_portfolio_value>self.current_enter_portfoliovalue*(1+self.longtakeprofit)):
#       self.exit_long(index)
#       return 1

#     elif(self.current_portfolio_value<self.current_maximum_portfolio_value*(1-self.longtrallingstoploss)):
#       self.exit_long(index)
#       return 1

#     elif(self.current_portfolio_value>self.current_maximum_portfolio_value*(1+self.longtrallingtakeprofit)):
#       self.exit_long(index)
#       return 1

#     elif(self.roll3[index]<self.roll22[index]-self.longatr*self.df['ATR'][index]):
#       self.exit_long(index)
#       return 1

#     # elif(self.df['trend'][i]<0):
#     #   print('asa')
#     #   self.exit_long(index)
#     #   return 1
#     return 0



#   def check_short_exitcondition(self,index):

#     self.current_portfolio_value=self.capital+self.current_quantity*self.close[index]
#     if(self.current_portfolio_value<self.current_enter_portfoliovalue*(1-self.shortstoploss)):
#       self.exit_short(index)
#       return 1
#     elif(self.current_portfolio_value>self.current_enter_portfoliovalue*(1+self.shorttakeprofit)):
#       self.exit_short(index)
#       return 1

#     elif(self.current_portfolio_value<self.current_maximum_portfolio_value*(1-self.shorttrallingstoploss)):
#       self.exit_short(index)
#       return 1

#     elif(self.current_portfolio_value>self.current_maximum_portfolio_value*(1+self.shorttrallingtakeprofit)):
#       self.exit_short(index)
#       return 1

#     elif(self.roll3[index]>self.roll22[index]+self.shortatr*self.df['ATR'][index]):
#       self.exit_short(index)
#       return 1

#     # elif(self.df['trend'][i]>0):
#     #   print('asa')
#     #   self.exit_short(index)
#     #   return 1


#     return 0



#   def backtest(self):
#     self.calculate_atr()
#     for i in range(len(self.df)):

#       if self.current_position==0:

#         if self.signal[i]==0:
#           self.update_noposition(i)

#         elif self.signal[i]==1:
#           self.enter_long(i)

#         elif self.signal[i]==-1:
#           self.enter_short(i)

#       elif self.current_position==1:
#         if self.signal[i]==-1:
#           self.exit_long(i)

#         elif self.check_long_exitcondition(i)==0:
#           self.update_longposition(i)

#       else:
#         if self.signal[i]==1:
#           self.exit_short(i)

#         elif self.check_short_exitcondition(i)==0:
#           self.update_shortposition(i)


#     if self.current_position==1:
#       self.exit_long(len(self.df)-1)

#     elif self.current_position==-1:
#       self.exit_short(len(self.df)-1)


#   def tradewise_log(self):
#     self.tradelog=pd.DataFrame()
#     self.tradelog['entertime']=self.entry
#     self.tradelog['exittime']=self.exit
#     self.tradelog['holdingtime']=(self.tradelog['exittime']-self.tradelog['entertime'])
#     self.tradelog['Drawdown']=self.drawdown
#     self.tradelog['Dip']=self.dip
#     self.tradelog['Returns']=self.returns
#     self.tradelog['tradetype']=self.tradetype
#     self.tradelog['quantity']=self.quantity_tradwise
#     return self.tradelog

#   def calculate_ratios(self):

#     plt.subplot(2, 1, 1)
#     plt.plot(self.portfolio_value)
#     plt.title('Portfolio Value')

#     self.profit = self.capital - 1000000
#     print(f"Profit: {self.profit}")

#     self.returns = self.profit / 10_000
#     print(f"Returns: {self.returns}")

#     self.wintrade = len(self.tradelog[self.tradelog['Returns'] > 0])
#     print(f"Winning Trades: {self.wintrade}")

#     self.losstrade = len(self.tradelog[self.tradelog['Returns'] < 0])
#     print(f"Losing Trades: {self.losstrade}")

#     if(self.losstrade != 0):
#       self.winlossration = self.wintrade / self.losstrade
#     else:
#       self.winlossration = float('inf')
#     # self.winlossration = self.wintrade / self.losstrade if self.losstrade != 0 else float('inf')
#     print(f"Win/Loss Ratio: {self.winlossration}")

#     self.maxdrawdown = self.tradelog['Drawdown'].max()
#     print(f"Max Drawdown: {self.maxdrawdown}")

#     self.benchmark_return = (((1000000/self.close[0]) * self.close[len(self.close) - 1] - 1000000))
#     print(f"Benchmark Returns: {self.benchmark_return/10000}")

#     self.avgdrawdown = self.tradelog['Drawdown'].mean()
#     print(f"Average Drawdown: {self.avgdrawdown}")

#     self.maxdip = self.tradelog['Dip'].max()
#     print(f"Max Dip: {self.maxdip}")

#     self.avgdip = self.tradelog['Dip'].mean()
#     print(f"Average Dip: {self.avgdip}")

#     self.maxholdingtime = self.tradelog['holdingtime'].max()
#     print(f"Max Holding Time: {self.maxholdingtime}")

#     self.avgholdingtime = self.tradelog['holdingtime'].mean()
#     print(f"Average Holding Time: {self.avgholdingtime}")

#     self.maxprofit = self.tradelog['Returns'].max()
#     print(f"Max Profit: {self.maxprofit}")

#     self.avgprofit = self.tradelog['Returns'].mean()
#     print(f"Average Profit: {self.avgprofit}")

#     self.dailyreturns = np.diff(self.portfolio_value) * 100 / self.portfolio_value[:-1]
#     # print(f"Daily Returns: {self.dailyreturns}")

#     self.daily_volatility = self.dailyreturns.std()
#     # print(f"Daily Volatility: {self.daily_volatility}")

#     self.annual_volatility = self.daily_volatility * np.sqrt(252)
#     # print(f"Annual Volatility: {self.annual_volatility}")

#     self.annual_volatility_negative = self.dailyreturns[self.dailyreturns < 0].std() * np.sqrt(252)
#     # print(f"Annual Volatility (Negative): {self.annual_volatility_negative}")

#     self.annual_excess_return = (self.dailyreturns.mean() * 252 - 0.03)
#     # print(f"Annual Excess Return: {self.annual_excess_return}")

#     self.sharpe = self.annual_excess_return / self.annual_volatility
#     print(f"Sharpe Ratio: {self.sharpe}")

#     self.sortino = self.annual_excess_return / self.annual_volatility_negative
#     print(f"Sortino Ratio: {self.sortino}")










In [54]:
from multiprocessing import current_process
# @title Backtesting engine
import plotly.graph_objects as go
class Tradingstrat_used:


  def __init__(self,df,Key_parameters):

    self.df = df  # The DataFrame containing market data or stock prices for analysis or trading.
    self.capital = 1000000  # Initial capital for trading, set to 1,000,000 units of currency.
    self.percentage = Key_parameters[0]  # Maximum percentage of the capital to risk in a single trade (35%).
    self.short_percentage = Key_parameters[1]  # Percentage of capital to allocate for short positions, set to 1%.
    self.current_position = []  # List to track the current positions in the portfolio (e.g., stocks held).
    self.current_quantity = []  # List to track the quantity of assets currently held.
    self.portfolio_value = []  # List to track the historical portfolio values over time.
    self.position = []  # Another list to track the current open positions, might be redundant with `self.current_position`.


    self.current_portfolio_value = 1000000  # The current value of the entire portfolio, initialized to the starting capital of 1,000,000.
    self.current_maximum_portfolio_value = 1000000  # The highest value the portfolio has reached so far, initialized to the starting capital
    self.current_minimum_portfolio_value = 1000000  # The lowest value the portfolio has reached so far, initialized to the starting capital.



    self.current_maximum_drawdown = 0  # The maximum drawdown observed so far, initialized to 0. This represents the largest drop from a peak to a trough in the portfolio value.
    self.current_maximum_dip = 0  # The maximum dip in the portfolio value, initialized to 0. This represents the largest decrease in value from a recent high.
    self.close = df['Close']  # Series or column of closing prices from the DataFrame.
    self.signal = df['signals']  # Series or column of trading signals from the DataFrame.
    self.pvalue = []  # List to track p-values, which are used in statistical tests.
    self.current_enterprice = []  # List to track the entry prices for trades.
    self.current_maxprice=[]
    self.margin = []  # List to track margin requirements or margin used for trading.
    self.entry = []  # List to record the entry points for trades.
    self.exit = []  # List to record the exit points for trades.
    self.quantity_tradwise = []  # List to track quantities of assets traded on a per-trade basis.
    self.tradetype = []  # List to record the type of trades (e.g., buy or sell).
    self.returns = []  # List to track returns from trades or portfolio performance.

    self.new_signals = []  # List to store new trading signals that might be generated or processed.
    self.longstoploss = Key_parameters[2] # Stop loss threshold for long positions, set at 2% below the entry price.
    self.longtakeprofit = Key_parameters[3]  # Take profit threshold for long positions, set at 4% above the entry price.
    self.longtrallingstoploss=Key_parameters[4] #Tralling take profit for long positions
    self.shortstoploss = Key_parameters[5]  # Stop loss threshold for short positions, set at 1% above the entry price.
    self.shorttakeprofit = Key_parameters[6]  # Take profit threshold for short positions, set at 4% below the entry price
    self.shorttrallingstoploss=Key_parameters[7] #Tralling take profit for short positions,
    self.current_entry = []  # List to record the entry points for the current trades.
    self.current_tradetype = []  # List to record the type of trades (e.g., long or short) for the current trades.
    self.current_quantity_tradwise = []  # List to track quantities of assets traded on a per-trade basis for the current trades.
    self.current_enterprice = []  # List to track the entry prices for the current trades.



  def enter_long(self, index):
      # Update the current portfolio value based on the existing positions.
      self.current_portfolio_value = self.capital
      for iii in range(len(self.current_position)):
          if self.current_position[iii] == 1:
              # If the current position is long, add the value of the position to the portfolio value.
              self.current_portfolio_value += self.current_quantity[iii] * self.close[index]
          elif self.current_position[iii] == -1:
              # If the current position is short, add the margin and the profit/loss from the short position to the portfolio value.
              self.current_portfolio_value += self.margin[iii]
              self.current_portfolio_value += self.current_quantity[iii] * (self.current_enterprice[iii] - self.close[index])

      # Add a new long position to the portfolio.
      self.current_position.append(1)
      # Calculate the quantity to buy based on the percentage of the portfolio value and the closing price.
      self.current_quantity.append(min(self.capital, self.current_portfolio_value * self.percentage) // self.close[index])
      # Update the capital by subtracting the cost of the new long position.
      self.capital -= self.close[index] * self.current_quantity[-1]
      # Record the entry price of the new long position.
      self.current_enterprice.append(self.close[index])
      # Append 0 to the margin list as no margin is used for this long position.
      self.margin.append(0)

      # Record the index of the trade, type of trade, and the signal.
      self.current_entry.append(index)
      self.current_maxprice.append(self.close[index])
      self.current_tradetype.append('Long')
      self.new_signals.append(1)
      # Append the updated portfolio value and position to their respective lists.
      self.portfolio_value.append(self.current_portfolio_value)
      self.position.append(self.current_position)



  def enter_short(self, index):
      # Check if there is enough capital to open a new short position.
      if self.capital < self.close[index]:
          return

      # Update the current portfolio value based on the existing positions.
      self.current_portfolio_value = self.capital
      for iii in range(len(self.current_position)):
          if self.current_position[iii] == 1:
              # If the current position is long, add the value of the position to the portfolio value.
              self.current_portfolio_value += self.current_quantity[iii] * self.close[index]
          elif self.current_position[iii] == -1:
              # If the current position is short, add the margin and the profit/loss from the short position to the portfolio value.
              self.current_portfolio_value += self.margin[iii]
              self.current_portfolio_value += self.current_quantity[iii] * (self.current_enterprice[iii] - self.close[index])

      # Add a new short position to the portfolio.
      self.current_position.append(-1)
      # Calculate the quantity to short based on the percentage of the portfolio value and the stop loss threshold.
      self.sq = min(self.capital, self.current_portfolio_value * self.short_percentage) // (self.close[index] * self.shortstoploss)
      self.current_quantity.append(self.sq)
      # Deduct the cost of the short position from capital.
      self.capital -= min(self.capital, self.current_portfolio_value * self.short_percentage)
      # Record the margin used for the short position.
      self.margin.append(min(self.capital, self.current_portfolio_value * self.short_percentage))
      # Record the entry price of the new short position.
      self.current_enterprice.append(self.close[index])

      # Record the index of the trade, type of trade, and the signal.
      self.current_entry.append(index)
      self.current_maxprice.append(self.close[index])
      self.current_tradetype.append('Short')
      self.new_signals.append(-1)
      # Append the updated portfolio value and position to their respective lists.
      self.portfolio_value.append(self.current_portfolio_value)
      self.position.append(self.current_position)



  def exit_long(self, index, iter):
      # Update the capital by adding the value received from closing the long position.
      self.capital += self.close[index] * self.current_quantity[iter]

      # Log the trade details.
      self.entry.append(self.current_entry[iter])  # Record the entry index of the trade.
      self.exit.append(index)  # Record the exit index of the trade.
      self.tradetype.append(self.current_tradetype[iter])  # Record the type of trade (long).
      self.current_quantity_tradwise.append(self.current_quantity[iter])  # Record the quantity of the asset traded.
      self.portfolio_value.append(self.current_portfolio_value)  # Record the portfolio value at the time of exit.
      self.new_signals.append(-1)  # Append a signal indicating that the long position has been exited.
      # Calculate and record the returns from the trade.
      self.returns.append((self.close[index] - self.close[self.current_entry[iter]]) * self.current_quantity[iter])

      # Remove the closed long position details from the lists.
      self.current_position.pop(iter)  # Remove the position type (long) at index `iter`.
      self.margin.pop(iter)  # Remove the margin used for the position.
      self.current_quantity.pop(iter)  # Remove the quantity of the asset held.
      self.current_enterprice.pop(iter)  # Remove the entry price of the position.
      self.current_tradetype.pop(iter)  # Remove the trade type (long) at index `iter`.
      self.current_entry.pop(iter)  # Remove the entry index of the trade.
      self.current_maxprice.pop(iter) #Remove the maximum price during the trade


  def exit_short(self, index, iter):
      # Update the capital by adding the margin and the profit from closing the short position.
      self.capital += self.margin[iter]
      self.capital += (self.close[self.current_entry[iter]] - self.close[index]) * self.current_quantity[iter]

      # Log the trade details.
      self.entry.append(self.current_entry[iter])  # Record the entry index of the trade.
      # Calculate and record the returns from the short trade.
      self.returns.append((self.current_enterprice[iter] - self.close[index]) * self.current_quantity[iter])
      self.exit.append(index)  # Record the exit index of the trade.
      self.current_quantity_tradwise.append(self.current_quantity[iter])  # Record the quantity of the asset traded.
      self.tradetype.append(self.current_tradetype[iter])  # Record the type of trade (short).
      self.portfolio_value.append(self.current_portfolio_value)  # Record the portfolio value at the time of exit.
      self.new_signals.append(1)  # Append a signal indicating that the short position has been exited.

      # Remove the closed short position details from the lists.
      self.current_position.pop(iter)  # Remove the position type (short) at index `iter`.
      self.current_quantity.pop(iter)  # Remove the quantity of the asset held.
      self.margin.pop(iter)  # Remove the margin used for the position.
      self.current_enterprice.pop(iter)  # Remove the entry price of the position.
      self.current_entry.pop(iter)  # Remove the entry index of the trade.
      self.current_maxprice.pop(iter) #Remove the maximum price during the trade
      self.current_tradetype.pop(iter)  # Remove the trade type (short) at index `iter`.


  def current_drawdown(self):
      # Calculate the current drawdown as a percentage.
      return (self.current_maximum_portfolio_value - self.current_portfolio_value) * 100 / self.current_maximum_portfolio_value

  def current_dip(self):
      # Calculate the current dip as a percentage relative to a fixed initial value of 1,000,000.
      return (1000000 - self.current_portfolio_value) * 100 / 1000000


  def update_riskmanagment(self,index):

    self.strat1=[0.5,0.001,0.03,0.06,0.03,0.02,0.04,0.015]
    self.strat2=[0.5,0.001,0.03,0.06,0.03,0.02,0.04,0.015]
    self.strat3=[0.5,0.001,0.03,0.06,0.03,0.02,0.04,0.015]
    self.strat4=[0.5,0.001,0.03,0.06,0.03,0.02,0.04,0.015]
    self.strat5=[0.5,0.001,0.03,0.06,0.03,0.02,0.04,0.015]
    self.strat6=[0.5,0.001,0.03,0.06,0.03,0.02,0.04,0.015]
    self.strat7=[0.5,0.001,0.03,0.06,0.03,0.02,0.04,0.015]
    self.strat8=[0.5,0.001,0.03,0.06,0.03,0.02,0.04,0.015]
    self.strat9=[0.5,0.001,0.03,0.06,0.03,0.02,0.04,0.015]

    if(self.df['strat_used'][index]==1):
      a=self.strat1
    elif (self.df['strat_used'][index]==2):
      a=self.strat2
    elif (self.df['strat_used'][index]==3):
      a=self.strat3
    elif (self.df['strat_used'][index]==4):
      a=self.strat4
    elif (self.df['strat_used'][index]==5):
      a=self.strat5
    elif (self.df['strat_used'][index]==6):
      a=self.strat6
    elif (self.df['strat_used'][index]==7):
      a=self.strat7
    elif (self.df['strat_used'][index]==8):
      a=self.strat8
    elif (self.df['strat_used'][index]==9):
      a=self.strat9

    self.percentage=a[0]
    self.short_percentage=a[1]
    self.longstoploss = a[2]
    self.longtakeprofit = a[3]
    self.longtrallingstoploss=a[4]
    self.shortstoploss = a[5]
    self.shorttakeprofit = a[6]
    self.shorttrallingstoploss= a[7]




  def update_values(self, index):
      # Update the current portfolio value based on the capital and positions.
      self.current_portfolio_value = self.capital

      for iii in range(len(self.current_position)):
          if self.current_position[iii] == 1:
              # If the current position is long, add the value of the position at the current close price.
              self.current_portfolio_value += self.current_quantity[iii] * self.close[index]
          elif self.current_position[iii] == -1:
              # If the current position is short, add the margin and profit/loss from the short position.
              self.current_portfolio_value += self.margin[iii]
              self.current_portfolio_value += self.current_quantity[iii] * (self.current_enterprice[iii] - self.close[index])

      for iii in range(len(self.current_position)):
        if(self.current_position[iii]==1):
          self.current_maxpirce=max(self.current_maxprice[iii],self.close[index])
        elif(self.current_position[iii]==-1):
          self.current_maxpirce=min(self.current_maxprice[iii],self.close[index])

      # Append the updated portfolio value to the list of values.
      self.pvalue.append(self.current_portfolio_value)

      # Update the maximum portfolio value observed.
      self.current_maximum_portfolio_value = max(self.current_portfolio_value, self.current_maximum_portfolio_value)
      # Update the maximum drawdown observed.
      self.current_maximum_drawdown = max(self.current_maximum_drawdown, self.current_drawdown())
      # Update the maximum dip observed.
      self.current_maximum_dip = max(self.current_maximum_dip, self.current_dip())



  def check_long_exitcondition(self, index, i):
      # Check if the current close price is below the stop loss threshold.
      if self.close[index] < self.current_enterprice[i] * (1 - self.longstoploss):
          # Exit the long position if the stop loss condition is met.
          self.exit_long(index, i)
          return 1

      # Check if the current close price is above the take profit threshold.
      elif self.close[index] > self.current_enterprice[i] * (1 + self.longtakeprofit):
          # Exit the long position if the take profit condition is met.
          self.exit_long(index, i)
          return 1

      elif self.close[index]<self.current_maxprice[i]*(1-self.longtrallingstoploss):
        self.exit_long(index,i)
        return 1

      # Return 0 if neither condition is met.
      return 0


  def check_short_exitcondition(self, index, i):
      # Check if the current close price is above the stop loss threshold for a short position.
      if self.close[index] > self.current_enterprice[i] * (1 + self.shortstoploss):
          # Exit the short position if the stop loss condition is met.
          self.exit_short(index, i)
          return 1

      # Check if the current close price is below the take profit threshold for a short position.
      elif self.close[index] < self.current_enterprice[i] * (1 - self.shorttakeprofit):
          # Exit the short position if the take profit condition is met.
          self.exit_short(index, i)
          return 1


      elif self.close[index]>self.current_maxprice[i]*(1+self.shorttrallingstoploss):
        self.exit_short(index, i)
        return 1

      # Return 0 if neither condition is met.
      return 0



  def backtest(self):
      # Iterate through all the data points in the DataFrame.
      for i in range(len(self.df)):
          # Update the risk management based on the current data point.
          self.update_riskmanagment(i)
          # Enter a long position if the signal indicates a buy (1).
          if self.signal[i] == 1:
              self.enter_long(i)
          # Enter a short position if the signal indicates a sell (-1).
          if self.signal[i] == -1:
              self.enter_short(i)

          ii = 0
          # Iterate through the current positions to check for exit conditions.
          while ii < len(self.current_position):
              if self.current_position[ii] == 1:
                  # Check if the long position exit condition is met.
                  if self.check_long_exitcondition(i, ii) == 0:
                      ii += 1

              elif self.current_position[ii] == -1:
                  # Check if the short position exit condition is met.
                  if self.check_short_exitcondition(i, ii) == 0:
                      ii += 1

          # Update the portfolio values after processing the signals and positions.
          self.update_values(i)

      # After processing all data points, ensure all remaining positions are closed.
      while self.current_position != []:
          if self.current_position[0] == 1:
              # Exit the first long position.
              self.exit_long(len(self.close) - 1, 0)
          elif self.current_position[0] == -1:
              # Exit the first short position.
              self.exit_short(len(self.close) - 1, 0)



  def tradewise_log(self):
      # Initialize an empty DataFrame for trade logs.
      self.tradelog = pd.DataFrame()

      # Populate the DataFrame with trade log data.
      self.tradelog['entertime'] = self.entry  # Entry timestamps or indices for trades.
      self.tradelog['exittime'] = self.exit    # Exit timestamps or indices for trades.

      # Calculate the holding time for each trade.
      self.tradelog['holdingtime'] = self.tradelog['exittime'] - self.tradelog['entertime']

      # Record the returns for each trade.
      self.tradelog['Returns'] = self.returns

      # Record the type of each trade (long/short).
      self.tradelog['tradetype'] = self.tradetype

      # Record the quantity of each trade.
      self.tradelog['quantity'] = self.current_quantity_tradwise


      return self.tradelog

  def calculate_ratios(self):
      # Generate trade log DataFrame.
      self.tradewise_log()

      fig = go.Figure()

      fig.add_trace(go.Scatter(y=self.pvalue, mode='lines', name='Portfolio Value'))

      fig.update_layout(
          title='Portfolio Value',
          xaxis_title='Time',
          yaxis_title='Value'
      )



      # Calculate profit based on the final capital compared to initial capital.
      self.profit = self.capital - 1000000
      print(f"Profit: {self.profit}")

      # Calculate returns as a proportion of the initial capital.
      self.returns = self.profit / 10_000
      print(f"Returns: {self.returns}")

      # Calculate number of winning trades.
      self.wintrade = len(self.tradelog[self.tradelog['Returns'] > 0])
      print(f"Winning Trades: {self.wintrade}")

      # Calculate number of losing trades.
      self.losstrade = len(self.tradelog[self.tradelog['Returns'] < 0])
      print(f"Losing Trades: {self.losstrade}")

      # Calculate win/loss ratio, handle division by zero.
      if self.losstrade != 0:
          self.winlossration = self.wintrade / self.losstrade
      else:
          self.winlossration = float('inf')  # Infinite if there are no losing trades.
      print(f"Win/Loss Ratio: {self.winlossration}")

      # Print maximum drawdown observed.
      print(f"Max Drawdown: {self.current_maximum_drawdown}")

      # Calculate benchmark return based on initial and final close values.
      self.benchmark_return = (((1000000 / self.close[0]) * self.close[len(self.close) - 1] - 1000000))
      print(f"Benchmark Returns: {self.benchmark_return / 10_000}")

      # Print maximum dip observed.
      print(f"Max Dip: {self.current_maximum_dip}")

      # Calculate maximum holding time for trades.
      self.maxholdingtime = self.tradelog['holdingtime'].max()
      print(f"Max Holding Time: {self.maxholdingtime}")

      # Calculate average holding time for trades.
      self.avgholdingtime = self.tradelog['holdingtime'].mean()
      print(f"Average Holding Time: {self.avgholdingtime}")

      # Calculate maximum profit for a single trade.
      self.maxprofit = self.tradelog['Returns'].max()
      print(f"Max Profit: {self.maxprofit}")

      # Calculate average profit across all trades.
      self.avgprofit = self.tradelog['Returns'].mean()
      print(f"Average Profit: {self.avgprofit}")

      # Calculate daily returns as percentage change in portfolio value.
      self.dailyreturns = np.diff(self.pvalue) * 100 / self.pvalue[:-1]

      # Calculate daily volatility as standard deviation of daily returns.
      self.daily_volatility = self.dailyreturns.std()

      # Annualize the daily volatility.
      self.annual_volatility = self.daily_volatility * np.sqrt(252)

      # Annualize the volatility of negative daily returns.
      self.annual_volatility_negative = self.dailyreturns[self.dailyreturns < 0].std() * np.sqrt(252)

      # Calculate annual excess return (subtracting risk-free rate of 3%).
      self.annual_excess_return = (self.dailyreturns.mean() * 252 - 0.03)

      # Calculate Sharpe ratio (return per unit of risk).
      self.sharpe = self.annual_excess_return / self.annual_volatility
      print(f"Sharpe Ratio: {self.sharpe}")

      # Calculate Sortino ratio (return per unit of negative risk).
      self.sortino = self.annual_excess_return / self.annual_volatility_negative
      print(f"Sortino Ratio: {self.sortino}")

      fig.show() # Title of the portfolio value plot.

In [58]:
def backtesting(ticker,start_date,end_date):
    stock,final_signals,strat_used=get_your_signals(ticker,start_date,end_date)
    Key_parameters=[0.5,0.001,0.03,0.06,0.03,0.02,0.04,0.015]
    stock["signals"]=final_signals
    stock["datetime"]=stock.index
    stock["strat_used"]=strat_used
    obj2=Tradingstrat_used(stock, Key_parameters)
    obj2.backtest()
    trade_log=obj2.tradewise_log()
    obj2.calculate_ratios()
    return trade_log, stock
# stock,final_signals,strat_used=get_your_signals(ticker,start_date,end_date)
# stock["signals"]=final_signals
# stock["datetime"]=stock.index
# stock["strat_used"]=strat_used
ticker=input("Enter the ticker")
start_date=input("Enter the start date")
end_date=input("Enter the end date")
trade_log, stock=backtesting(ticker,start_date,end_date)
print(trade_log)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


2017-05-31 00:00:00 RELIANCE.NS


[*********************100%%**********************]  1 of 1 completed


15


[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed


2020-07-31 00:00:00 ONGC.NS


[*********************100%%**********************]  1 of 1 completed


1245
Profit: 515121.85284230136
Returns: 51.512185284230135
Winning Trades: 149
Losing Trades: 217
Win/Loss Ratio: 0.6866359447004609
Max Drawdown: 28.030373248039233
Benchmark Returns: 182.64300752162168
Max Dip: 4.128139714111341
Max Holding Time: 50
Average Holding Time: 9.026097271648872
Max Profit: 87258.36511230469
Average Profit: 613.1152143919595
Sharpe Ratio: 0.47291985294021477
Sortino Ratio: 0.6678873572409699


     entertime  exittime  holdingtime       Returns tradetype  quantity
0           11        16            5  32639.492188      Long     592.0
1           13        16            3  34456.760132      Long     593.0
2           16        20            4    -30.035828      Long       1.0
3           17        23            6 -34183.261780      Long     603.0
4           18        23            5   3376.633301     Short      60.0
..         ...       ...          ...           ...       ...       ...
838       1221      1230            9 -33471.914795      Long     327.0
839       1222      1230            8     -0.000000      Long       0.0
840       1226      1230            4     -0.000000      Long       0.0
841       1227      1230            3     -0.000000      Long       0.0
842       1209      1235           26     -0.000000      Long       0.0

[843 rows x 6 columns]


In [66]:
stock[stock['strat_used'] == 7]

,Open,High,Low,Close,Adj Close,Volume,datetime,signals,strat_used
Date,,,,,,,,,
2018-01-22,866.787415,890.834351,859.472778,888.274231,861.711365,22850366,2018-01-22,0,7
2018-01-23,891.474426,906.057983,891.474426,899.017639,872.133362,11263405,2018-01-23,1,7
2018-01-24,897.646118,897.646118,880.182373,883.565430,857.143250,7144742,2018-01-24,1,7
2018-01-25,884.982605,888.731384,874.239197,883.153992,856.744080,6834950,2018-01-25,-1,7
2018-01-29,883.428284,893.577332,877.119385,881.873901,855.502319,4948532,2018-01-29,-1,7
...,...,...,...,...,...,...,...,...,...
2022-12-16,2373.032227,2417.151611,2361.171631,2368.047852,2351.730469,7918674,2022-12-16,-1,7
2022-12-19,2382.262207,2403.491211,2369.063232,2399.153076,2382.621338,3864966,2022-12-19,1,7
2022-12-20,2384.938965,2423.797119,2368.417236,2419.920654,2403.246094,3733794,2022-12-20,1,7


In [ ]:
obj2.pvalue

[1000000,
 1000000,
 1000000,
 1000000,
 1000000,
 1000000,
 1000000,
 1000000,
 1000000,
 1000000,
 1000000.0,
 1007919.0422973633,
 1010695.4191894531,
 1010253.75,
 999905.3345947266,
 989935.5037841797,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1050006.75,
 1052780.5720825195,
 1054526.313293457,
 1042847.5720825195,
 1040238.4557495117,
 977486.2230834961,
 977486.2230834961,
 977486.2230834961,
 982133.4086303711,
 967252.8374633789,
 937558.655456543,
 937558.655456543,
 939359.8786010742,
 944155.4631958008,
 970341.9631958008,
 967209.5352172852,
 993974.3522338867,
 984823.2803955078,
 1020880.3118896484,
 1030828.7276611328,
 1022364.8431396484,
 1029333.3431396484,
 1022790.7587890625,
 1016459.2587890625,
 1015679.2587890625,
 1036447.2587890625,
 1038981.6416015625,
 1056792.8759765625,
 1065138.8854980469,


# Livetesting 

In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import time
from datetime import datetime
from datetime import timedelta
import re
import pandas

refresh_time = 5 # seconds


def call_new_function(df,ticker):
    loopback = 10
    final_signals = []
    # sector=nse[nse["SYMBOL"]==ticker]["Industry"].values[0]
    df["price"] = df["price"].replace(",", "", regex=True)
    df["Date"]=df["timestamp"]
# Convert the 'price' column to floats
    df["Close"] = df["price"].astype(float)
    refresh_rate=15
    x=30-refresh_rate
    
    

    # Printing DataFrame for debugging purposes
    print(df)

    # Ensure that df has enough data to process
    if len(df) > loopback:
        # Process the DataFrame
        df = {
    'sector':sector,
    'ticker':ticker,
    'Close': df['Close'].to_numpy(),
    
    
    
    'Date': df['Date'].to_numpy()
}
        df = give_parameters(df)
        mapp ,tick= mapping(df[0:loopback], ticker)
        
        print(mapp,tick)
        start_date = (mapp + relativedelta(days=-refresh_rate)).strftime('%Y-%m-%d')
        end_date = (mapp + relativedelta(days=+x)).strftime('%Y-%m-%d')
        data = yf.download(tick, start_date, end_date)
        data["Date"]=data.index
        data = {
    'sector':sector,
    'ticker':ticker,
    'Close': data['Close'].to_numpy(),
    'Open': data['Open'].to_numpy(),
    'High': data['High'].to_numpy(),
    'Low': data['Low'].to_numpy(),
    'Date': data['Date'].to_numpy()
        }
        data = give_parameters(data)
        # print(data)
        stratt=get_your_strategy(data)

        signals = signal_generator(df,stratt)
        # signals=[0,1]
        

        # Ensure signals is a list
        if not isinstance(signals, list):
            signals = signals.tolist()

        # Replace the initial signals with 0 for the loopback period
        signals[:loopback] = [0] * loopback
        final_signals = signals

        # Add the signals to the DataFrame
        df["signals"] = final_signals

        # Backtesting the strategy
        obj_live = Tradingstragegy(df)
        obj_live.backtest()

        # Assuming portfolio_value is updated by backtest method
        df["portfolio_value"] = obj_live.portfolio_value

        # Clear the previous output in the Jupyter notebook
        clear_output(wait=True)

        # Create subplots for the close price and portfolio value
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

        # Plot the close price data on the first subplot
        ax1.plot(df['timestamp'], df['Close'], color='blue', label='Close Price')
        ax1.set_title('Close Price Over Time')
        ax1.set_xlabel('Timestamp')
        ax1.set_ylabel('Close Price')
        ax1.legend(loc='upper left')
        ax1.grid(True)
        
        # Plot the portfolio value data on the second subplot
        ax2.plot(df['timestamp'], obj_live.portfolio_value, color='green', label='Portfolio Value')
        ax2.set_title('Portfolio Value Over Time')
        ax2.set_xlabel('Timestamp')
        ax2.set_ylabel('Portfolio Value')
        ax2.legend(loc='upper left')
        ax2.grid(True)

        # Adjust the layout to prevent overlap
        plt.tight_layout()

        # Display the plots
        plt.show()
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

    
    
    
    
    
    

    
    
    
    
    
    
    

    
    

# Display the plots

        # df=give_parameters(df)
        # mapp=mapping(df,ticker)
        # stratt=strat(mapp)
        # signals=signal_genrator(df,stratt)
        # if not isinstance(signals, list):
            # signals=signals.tolist()
        # signals[:loopback]=0
        # final_signals=signals
    
    
    # if len(df)>loopback:
        # 
        # df=give_parameters(df)
        # mapp=mapping(df[0:loopback],ticker)
        # stratt=strat(mapp)
        # signals=signal_genrator(df,stratt)
        # if not isinstance(signals, list):
            # signals=signals.tolist()
        # signals[:loopback]=0
        # final_signals=signals
    # else:
        # final_signals.append(0)
    # df["signals"]=final_signals
    # obj_live=Tradingstragegy_live_testing(df)
    # obj_live.backtest()
    # obj.portfolio_value
    # plt.plot(obj.portfolio_value)
    # plt.show()    
    # print(df)
    
    
    
    

    

    
    
    


    
    

    
        
        
        
    

def get_datetime(date_str):
    

    # Original string

    # Remove extra text
    cleaned_str = re.sub(r'\s*INDEXNSE \u00b7 Disclaimer', '', date_str)

    # Parse the datetime
    # Note: \u202f is a thin space character, which might be interpreted as a regular space


    date_str = cleaned_str.replace('\u202f', ' ')
    # print(date_str)
    separator_index = date_str.find('\u00b7')

    # Chop the string from '\u00b7' onwards
    if separator_index != -1:
        date_str = date_str[:separator_index].strip()
    else:
        date_str = date_str.strip()

    # date_str = "Sep 2, 3:32:03\u202fPM GMT+5:30 \u00b7 INDEXNSE \u00b7 Disclaimer"

    # Find the index of 'AM' or 'PM'
    am_index = date_str.find('AM')
    pm_index = date_str.find('PM')

    # Determine the end index for slicing
    if am_index != -1:
        end_index = am_index + 2  # Include 'AM'
    elif pm_index != -1:
        end_index = pm_index + 2  # Include 'PM'
    else:
        end_index = len(date_str)  # No 'AM' or 'PM' found, keep the whole string

    # Slice the string up to the end index
    date_str = date_str[:end_index].strip()

    # print(date_str)

    date_obj = datetime.strptime(date_str, '%b %d, %I:%M:%S %p')

    # Adjust for the GMT offset manually, as strptime doesn't handle timezone offsets
    # The parsed datetime object will be naive; let's apply the GMT+5:30 offset
    

    # Extract the offset
    gmt_offset = '+5:30'
    hours, minutes = map(int, gmt_offset.split(':'))
    offset = timedelta(hours=hours, minutes=minutes)

# Apply the offset to the naive datetime object
    return date_obj
    date_obj = date_obj - offset
    

# Function to extract price from HTML
def get_price(soup_element):
    price = soup_element.find('main').find('div', 'AHmHk').get_text()
    return price

def get_timestamp(soup_element):
    timestamp = soup_element.find('main').find('div', 'ygUjEc').get_text()
    # print(timestamp)
    # print(get_datetime(timestamp))
    return timestamp

# Function to extract price change percentage from HTML
def get_change_percentage(soup_element):
    change_percentage_element = soup_element.find('main').find('div', 'JwB6zf')
    if change_percentage_element:
        return change_percentage_element.get_text()
    else:
        return None

# Function to extract stock title from HTML
def get_stock_title(soup_element):
    title = soup_element.find('main').find('div', 'zzDege').get_text()
    return title

# Function to extract stock description from HTML
def get_stock_description(soup_element):
    # Extracting detailed stock description
    description_elements = soup_element.find_all("div", {"class": "gyFHrc"})
    stock_description = {}

    for element in description_elements:
        description = element.find("div", {"class": "mfs7Fc"}).get_text()
        value = element.find("div", {"class": "P6K39c"}).get_text()
        stock_description[description] = value

    return stock_description

# Function to get HTML content from Google Finance URL
def get_finance_html(url):
    response = requests.get(url)
    return response.text

# Function to extract finance information from HTML
def get_price_timestamp(html):
    soup = BeautifulSoup(html, 'html.parser')
    price = get_price(soup)
    timestamp = get_timestamp(soup)
    return price, timestamp
def extract_finance_information_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    price = get_price(soup)
    change_percentage = get_change_percentage(soup)
    title = get_stock_title(soup)
    timestamp = get_timestamp(soup)
    description = get_stock_description(soup)
    finance_data = {
        'title': title,
        'price': price,
        'change_percentage': change_percentage,
        'description': description,
        'timestamp': timestamp
    }
    return finance_data




def get_data(url):
    html_content = get_finance_html(url)
    finance_data = extract_finance_information_from_html(html_content)
    return finance_data
# Main function to extract data from multiple Google Finance URLs
def main(ticker='BTC-INR',old_len=0):
    urls = [
        # 'https://www.google.com/finance/quote/BTC-INR',
        'https://www.google.com/finance/quote/{ticker}:NSE'
    ]
    # url = 'https://www.google.com/finance/quote/ETH-INR'
    url = 'https://www.google.com/finance/quote/{ticker}:NSE'
    finance_results = []
    finance_results_dict = {}
    
    # for url in urls:
        # finance_results_dict[url] = {'timestamp': [], 'price': []}
    iterations = 10
    finance_results_dict = {}
    
    # for i in range(iterations):
    i = 0
    while(1):
        i += 1
        price, timestamp = get_price_timestamp(get_finance_html(url))
        timestamp = get_datetime(timestamp)
        finance_results_dict[timestamp] = price
        if 1:
            df = pandas.DataFrame()
            df['timestamp'] = finance_results_dict.keys()
            df['price'] = finance_results_dict.values()
            len_=len(df)

            if len_>old_len:
                old_len=len_ 
                call_new_function(df,ticker)
                df.to_csv('finance_data.csv', index=False)
            time.sleep(refresh_time)
            print(i)
    
    
    
    
        
        
    
        


    # for i in range(iterations):
        # for url in urls:
            # print(i)
            # price, timestamp = get_price_timestamp(get_finance_html(url))
# 
            # 
            # timestamp_alt = get_datetime(timestamp)
# 
            # finance_results_dict[url][timestamp] = price
            # finance_results_dict[url].append((timestamp, price))
            # finance_results_dict[url]['timestamp'].append(timestamp)
            # finance_results_dict[url]['price'].append(price)
            # call_new_function()
        # if i != iterations - 1:
            # time.sleep(refresh_time)
        # html_content = get_finance_html(url)
        # finance_data = extract_finance_information_from_html(html_content)
        # finance_results.append(finance_data)
    # 
    # with open('finance_data.json', 'w') as f:
        # print("Dumping data")
        # json.dump(finance_results_dict, f, indent=4)

if __name__ == "__main__":
    y=input("Enter the stock name: ")
    main(y)
    # main()

            timestamp      price                Date      Close
0 1900-09-03 21:36:57  206170.64 1900-09-03 21:36:57  206170.64
1
2
3
4
5
6
            timestamp      price                Date      Close
0 1900-09-03 21:36:57  206170.64 1900-09-03 21:36:57  206170.64
1 1900-09-03 21:37:57  206182.49 1900-09-03 21:37:57  206182.49


ValueError: sample size is too short to use selected regression component